In [1]:
import json
from models.db_models import BasePromptTemplate, CharacterData
from llm_util import LLMProvider
from db_utils import DatabaseManager



def generate_character_data(llm_provider : LLMProvider, character_generation_pompt_template: BasePromptTemplate, db_manager: DatabaseManager, job_id: str) -> CharacterData:

    response_id = llm_provider.initiateResponse(character_generation_pompt_template.promptTemplate, model="gpt-4.1-nano")
    print(f"Submitted LLM request, response ID: {response_id}")

    response = llm_provider.getPooledResponse(response_id)
    ## find the first occurance in response.output where the type = 'message'
    response_text = ""
    if not response or not response.output:
        raise ValueError("Invalid response from LLM provider")
    for output in response.output:
        if output.type == "message" and output.content and len(output.content) > 0:
            response_text = output.content[0].text
            break


    print(f"Response: {response_text}")

    job = db_manager.get_job(job_id)
    if not job:
        raise ValueError(f"Job with ID {job_id} not found")
    
    job.characterData = CharacterData(data=response_text)
    db_manager.update_job_field(job_id, job)

    return job.characterData

In [2]:
from models.db_models import StoryverseMetaData, Job, CharacterData
import os
from dotenv import load_dotenv
from llm_util import OpenAiLLMProvider
from db_utils import DatabaseManager

if __name__ == "__main__":
    load_dotenv()
    
    # Initialize providers
    llm_provider = OpenAiLLMProvider()
    db_manager = DatabaseManager()
    
    # Get template
    story_verse = os.getenv('STORY_VERSE') or "SHERLOCK"
    meta_data = db_manager.get_meta_data(story_verse)

    ## do optional check of metadata
    if meta_data is None:
        raise ValueError(f"No metadata found for the given story verse: {story_verse}")
    meta_data = StoryverseMetaData(**meta_data.model_dump())
    
    # Create new job
    job = Job(
        storyVerse=story_verse,
        characterData=CharacterData(data=''),  # Provide an empty dict or appropriate default
        plot={},
        storyChain={},
        storySummary="",
        firstDraft="",
        climaxEnhancedStory="",
        finalStory=""
        )
    job_id = db_manager.create_job(job)
    
    print(f"Created job with ID: {job_id}")
    
    # STEP 1 - Generate Character Data
    characterData = generate_character_data(
        llm_provider,
        meta_data.characterGenearationPromptTemplate,
        db_manager,
        job_id)
    print(f"Updated job {job_id} with character data")
    
    # Close DB connection when done
    db_manager.close()

Created job with ID: 68bebf752ea30cc49afc93c5
Submitted LLM request, response ID: resp_68bebf75b158819fb98a6d7e1967328b046ae70c7d501698
######{'id': 'resp_68bebf75b158819fb98a6d7e1967328b046ae70c7d501698', 'object': 'response', 'created_at': 1757331317, 'status': 'queued', 'background': True, 'error': None, 'incomplete_details': None, 'instructions': None, 'max_output_tokens': None, 'max_tool_calls': None, 'model': 'gpt-4.1-nano-2025-04-14', 'output': [], 'parallel_tool_calls': True, 'previous_response_id': None, 'prompt_cache_key': None, 'reasoning': {'effort': None, 'summary': None}, 'safety_identifier': None, 'service_tier': 'auto', 'store': True, 'temperature': 1.0, 'text': {'format': {'type': 'text'}, 'verbosity': 'medium'}, 'tool_choice': 'auto', 'tools': [], 'top_logprobs': 0, 'top_p': 1.0, 'truncation': 'disabled', 'usage': None, 'user': None, 'metadata': {}}
###### here
######{'id': 'resp_68bebf75b158819fb98a6d7e1967328b046ae70c7d501698', 'object': 'response', 'created_at': 17